# Imports and data loading

In [1]:
%run Imports.ipynb
name = 'Kred' # Choose Kred or Mone

/home/RDC/fliegebe/.conda/envs/Work/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.__version__

'2.1.3'

In [3]:
# updated version of grid, because previous has 6'144 different combinations
param_grid = {
    'hidden_layer_sizes': [(h,) for h in [100, 200, 300]],
    'activation': ['relu'],                   # activation functions
    'solver': ['adam'],                                    # optimization algorithms
    'alpha': [0.0001, 0.01],                                # L2 regularization
    'learning_rate': ['constant'],                    # learning rate strategy
    'learning_rate_init': [0.005, 0.01, 0.05],                           # initial learning rate
    'max_iter': [100, 200, 500]                                        # training epochs
}

In [4]:
# specific imports
from sklearn.model_selection import ParameterGrid, cross_val_score, cross_val_predict
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    f1_score,
    roc_auc_score
)
from sklearn.neural_network import MLPClassifier

key_featsubgroups = pd.read_pickle('../pickle/2_FS/' + name + '/key_featsubgroups.pkl')
df = pd.read_pickle('../pickle/2_FS/' + name + '/2_df_new_.pkl')

In [5]:
print(df[target].value_counts()/df.shape[0])
print('df_shape: ', df.shape)

arrears
1   0.646
0   0.354
Name: count, dtype: float64
df_shape:  (129457, 418)


In [6]:
with open('../pickle/4_Model_Optuna/results_dict_updated.pkl', 'rb') as f:
    results_dict = pickle.load(f)

In [7]:
with open('../pickle/3_Model/xgb_default_params.pkl', 'rb') as f:
    default_params = pickle.load(f)


In [8]:
def split_data_4(df, key_featsubgroups=key_featsubgroups, target=target, test_size=0.2, random_state=42):
    """
    Splits the dataset into training and testing sets while excluding demographic features.

    Parameters:
    df (DataFrame): The dataset containing features and target variable.
    key_featsubgroups (DataFrame): A mapping of feature subgroups.
    target (str): The name of the target variable.
    test_size (float, optional): The proportion of the dataset to allocate for testing. Default is 0.2.
    random_state (int, optional): Random seed for reproducibility. Default is 42.

    Returns:
    tuple: X_train, X_test, y_train, y_test (training and testing datasets)
    """

    # Extract demographic features
    demo_feat = key_featsubgroups.loc[key_featsubgroups['subgroup'] == 'demo', 'list_features'].values[0]
    print("Demographic Features:", demo_feat)

    # Separate features (X) and target variable (y), excluding demographic features
    X = df.drop(columns=[target] + demo_feat)
    y = df[target]

    # Split the dataset into training (80%) and testing (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Print dataset shapes
    print(f"Training Features Shape: {X_train.shape}")
    print(f"Training Labels Shape: {y_train.shape}")
    print(f"Testing Features Shape: {X_test.shape}")
    print(f"Testing Labels Shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

In [9]:
x_train, x_test, y_train, y_test = split_data_4(df)

Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']
Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)


# 7) nn sklearn

In [10]:
def sklearn_nn(param_grid, df, results_dict):
    """
    Grid-search tunes and trains an MLPClassifier (one hidden layer),
    evaluates performance, and stores results in the given dictionary.

    Parameters:
    -----------
    param_grid : dict
        Grid of hyperparameters to try. Keys are param names for MLPClassifier
        (e.g. "hidden_layer_sizes", "activation", "solver", "learning_rate_init",
        "batch_size", "max_iter"), values are lists of candidate values.
    df : DataFrame
        Dataset containing features and the target variable.
    results_dict : dict
        Dictionary to store model performance metrics.

    Returns:
    --------
    best_params : dict
        Best hyperparameters found by grid search.
    results_dict : dict
        Updated results dictionary containing model evaluation metrics.
    """
    start_time = time.time()
    print("\nStarting grid-search for MLPClassifier...")

    # Split data
    X_train, X_test, y_train, y_test = split_data_4(df)

    # Build list of all param combinations
    grid = list(ParameterGrid(param_grid))
    print(f"Total parameter combinations to try: {len(grid)}")

    best_score = -np.inf
    best_params = None

    # Evaluate each combination via 5-fold CV accuracy
    for i, params in enumerate(grid, 1):
        print(f"Trying combination {i}/{len(grid)}: {params}")
        clf = MLPClassifier(**params, random_state=0)
        scores = cross_val_score(clf, X_train, y_train, cv=5, scoring="accuracy")
        mean_score = scores.mean()
        print(f" → mean CV accuracy: {mean_score:.4f}")
        if mean_score > best_score:
            best_score = mean_score
            best_params = params

    print("Best hyperparameters found:", best_params)
    print(f"Best CV accuracy: {best_score:.4f}")

    # Train final model on full training data
    clf_best = MLPClassifier(**best_params, random_state=0)
    clf_best.fit(X_train, y_train)

    # Obtain predictions
    y_train_pred = cross_val_predict(clf_best, X_train, y_train, cv=5)
    y_test_pred  = clf_best.predict(X_test)
    y_test_proba = clf_best.predict_proba(X_test)[:, 1]

    # Compute metrics
    cfm_train    = confusion_matrix(y_train, y_train_pred)
    cfm_test     = confusion_matrix(y_test,  y_test_pred)
    acc_train    = accuracy_score(y_train, y_train_pred)
    acc_test     = accuracy_score(y_test,  y_test_pred)
    f1_train_1   = f1_score(y_train, y_train_pred, pos_label=1)
    f1_train_0   = f1_score(y_train, y_train_pred, pos_label=0)
    f1_test_1    = f1_score(y_test,  y_test_pred,  pos_label=1)
    f1_test_0    = f1_score(y_test,  y_test_pred,  pos_label=0)
    roc_auc_test = roc_auc_score(y_test, y_test_proba)

    total_time = (time.time() - start_time) / 60
    print(f"Grid-search & training completed in {total_time:.2f} minutes")

    # Store results
    results_dict["sklearn_nn"] = {
        "classifier":             deepcopy(clf_best),
        "cfm_train":              cfm_train,
        "cfm_test":               cfm_test,
        "train_accuracy":         acc_train,
        "test_accuracy":          acc_test,
        "train F1-score label 1": f1_train_1,
        "train F1-score label 0": f1_train_0,
        "test F1-score label 1":  f1_test_1,
        "test F1-score label 0":  f1_test_0,
        "test roc auc score":     roc_auc_test,
        "param_grid":             param_grid,
        "best_params":            best_params,
        "time_m":                 total_time
    }

    return best_params, results_dict

In [11]:
best_params, results_dict = sklearn_nn(param_grid, df, results_dict)


Starting grid-search for MLPClassifier...
Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']
Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)
Total parameter combinations to try: 54
Trying combination 1/54: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.005, 'max_iter': 100, 'solver': 'adam'}
 → mean CV accuracy: 0.5372
Trying combination 2/54: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rat

In [19]:
with open('../pickle/Monedo_7/without_optuna/best_params.pkl','wb') as f: 
    pickle.dump(best_params, f, pickle.HIGHEST_PROTOCOL)

In [20]:
with open('../pickle/Monedo_7/without_optuna/results_dict.pkl','wb') as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)